# Stage 2 Baseline: MIL Bag Generation (50% Forgery) - 간단한 베이스라인

이 노트북은 Stage 1에서 추출한 ArcFace 임베딩을 이용해 Multiple Instance Learning(MIL) 학습을 위한 **간단한 베이스라인** Bag 데이터를 생성합니다. 

**핵심 단순화:**
- **단일 규칙**: Negative=한 작성자 14개 단어, Positive=두 작성자 7+7개 단어 랜덤 셔플
- **고정 윈도우**: (win=5, stride=1) → 10개 인스턴스 → 각 bag은 (10, 5, 256)
- **50/50 균형**: 각 split에서 정확히 50% Positive, 50% Negative 비율
- **복잡한 설정 제거**: 게이트, 커스텀 페어링, 스트레스 테스트 등 모든 복잡한 기능 제거

In [1]:
# 라이브러리 임포트 및 환경 설정
import os, random, pickle, numpy as np, pandas as pd
from collections import defaultdict

# GPU 설정 (선택적)
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"] = os.getenv('MIL_STAGE2_GPU', '1')

# 재현성을 위한 시드 설정
SEED_BASE = 42
np.random.seed(SEED_BASE)
random.seed(SEED_BASE)

print(f'환경 설정 완료: GPU={os.environ.get("CUDA_VISIBLE_DEVICES", "N/A")}, SEED={SEED_BASE}')

환경 설정 완료: GPU=1, SEED=42


In [2]:
# 경로 설정 및 데이터 로딩
embedding_dir = '/workspace/MIL/data/processed/embeddings'
raw_meta_csv  = '/workspace/MIL/data/raw/naver_ocr.csv'  # 선택 사항
bags_dir      = '/workspace/MIL/data/processed/bags'
os.makedirs(bags_dir, exist_ok=True)

margin_value = '0.4'
rng_global = np.random.default_rng(SEED_BASE)

# CSV 로딩 함수
def load_split_csv(split):
    csv_path = os.path.join(embedding_dir, f'mil_arcface_margin_{margin_value}_{split}_data.csv')
    df = pd.read_csv(csv_path)
    label_col = 'label' if 'label' in df.columns else 'author_id'
    emb_cols  = [c for c in df.columns if c.startswith('embedding')]
    assert len(emb_cols) > 0, "No embedding_* columns found."
    return df, label_col, emb_cols

# 데이터 로딩
train_df, label_col, emb_cols = load_split_csv('train')
val_df,   _,         _        = load_split_csv('val')
test_df,  _,         _        = load_split_csv('test')
embed_dim = len(emb_cols)

print(f"데이터 로딩 완료:")
print(f"  - Embedding 차원: {embed_dim}")
print(f"  - Label 컬럼: {label_col}")
print(f"  - Train: {len(train_df)}, Val: {len(val_df)}, Test: {len(test_df)}")

# 선택적 원본 메타데이터 로딩
try:
    original_df = pd.read_csv(raw_meta_csv)
    print(f"  - 원본 메타데이터: {len(original_df)} rows")
    has_raw_meta = True
except Exception:
    print(f"  - 원본 메타데이터: 없음")
    has_raw_meta = False

데이터 로딩 완료:
  - Embedding 차원: 256
  - Label 컬럼: label
  - Train: 208233, Val: 70533, Test: 72457
  - 원본 메타데이터: 556128 rows


In [3]:
# ==============================================================
# Stage 2 Baseline: MIL Bags (50% Forgery), Minimal & Reproducible
# ==============================================================

# 작성자별 인덱스 구축
def build_writer_index(df, label_col, emb_cols):
    w2 = {}
    for wid, g in df.groupby(label_col):
        w2[int(wid)] = {
            'emb': g[emb_cols].to_numpy(dtype=np.float32),
            'paths': g['path'].tolist() if 'path' in g.columns else [''] * len(g),
            'idx': g.index.to_list()
        }
    return w2

train_writers = build_writer_index(train_df, label_col, emb_cols)
val_writers   = build_writer_index(val_df,   label_col, emb_cols)
test_writers  = build_writer_index(test_df,  label_col, emb_cols)

def list_writer_ids(wdict): 
    return list(wdict.keys())

print(f"작성자 인덱스 구축 완료:")
print(f"  - Train writers: {len(train_writers)}")
print(f"  - Val writers: {len(val_writers)}")
print(f"  - Test writers: {len(test_writers)}")

# 샘플링 헬퍼 함수들
def sample_k(n, k, rng, replace_if_needed=True):
    """n개 중에서 k개 샘플링 (부족하면 중복 허용)"""
    if (not replace_if_needed) and n >= k:
        return rng.choice(n, size=k, replace=False).tolist()
    # 부족하면 중복 허용
    return rng.choice(n, size=k, replace=True).tolist()

def sliding_windows(seq, win=5, stride=1):
    """슬라이딩 윈도우로 시퀀스 분할"""
    # seq: list of tuples (emb, wid, path, orig_idx)
    windows, metas = [], []
    for i in range(0, len(seq) - win + 1, stride):
        chunk = seq[i:i+win]
        windows.append(np.stack([e for (e,_,_,_) in chunk], axis=0))  # (5, D)
        metas.append({
            'window_idx': i,
            'word_indices': [oi for (_,_,_,oi) in chunk],
            'word_paths':   [p  for (_,_,p, _) in chunk],
            'writer_ids':   [w  for (_,w,_, _) in chunk],
        })
    return windows, metas

def pack(words, wid, W):
    """단어 인덱스를 임베딩 튜플로 변환"""
    emb, paths, idxs = W['emb'], W['paths'], W['idx']
    return [(emb[w], wid, paths[w], idxs[w]) for w in words]

print("✓ 헬퍼 함수 정의 완료")

작성자 인덱스 구축 완료:
  - Train writers: 180
  - Val writers: 60
  - Test writers: 60
✓ 헬퍼 함수 정의 완료


In [4]:
# Bag 생성 함수 (베이스라인 규칙)
WIN = 5; STRIDE = 1; INSTANCES_PER_BAG = 10
TOK_NEG = 14; TOK_POS_A = 7; TOK_POS_B = 7

def make_negative_bag(wid, W, rng):
    """단일 작성자 Bag (레이블 0)"""
    emb, paths, idxs = W['emb'], W['paths'], W['idx']
    sel = sample_k(len(emb), TOK_NEG, rng, replace_if_needed=True)
    seq = pack(sel, wid, W)
    wins, metas = sliding_windows(seq, WIN, STRIDE)
    bag = np.stack(wins[:INSTANCES_PER_BAG], axis=0)  # (10, 5, D)
    return bag, metas[:INSTANCES_PER_BAG], [int(wid)]

def make_positive_bag(widA, widB, WA, WB, rng):
    """복수 작성자 Bag (레이블 1): A 7개 + B 7개 → 전체 셔플 → 윈도우"""
    embA, pathsA, idxA = WA['emb'], WA['paths'], WA['idx']
    embB, pathsB, idxB = WB['emb'], WB['paths'], WB['idx']
    
    selA = sample_k(len(embA), TOK_POS_A, rng, replace_if_needed=True)
    selB = sample_k(len(embB), TOK_POS_B, rng, replace_if_needed=True)
    
    seqA = [(embA[i], int(widA), pathsA[i], idxA[i]) for i in selA]
    seqB = [(embB[i], int(widB), pathsB[i], idxB[i]) for i in selB]
    
    seq = seqA + seqB
    rng.shuffle(seq)  # 핵심: 랜덤 셔플
    
    wins, metas = sliding_windows(seq, WIN, STRIDE)
    bag = np.stack(wins[:INSTANCES_PER_BAG], axis=0)
    return bag, metas[:INSTANCES_PER_BAG], [int(widA), int(widB)]

print("✓ Bag 생성 함수 정의 완료")
print(f"  - Negative: {TOK_NEG}개 단어 → {INSTANCES_PER_BAG}개 인스턴스")
print(f"  - Positive: {TOK_POS_A}+{TOK_POS_B}개 단어 셔플 → {INSTANCES_PER_BAG}개 인스턴스")
print(f"  - 윈도우: (win={WIN}, stride={STRIDE})")

✓ Bag 생성 함수 정의 완료
  - Negative: 14개 단어 → 10개 인스턴스
  - Positive: 7+7개 단어 셔플 → 10개 인스턴스
  - 윈도우: (win=5, stride=1)


In [5]:
# Split 생성 함수
def generate_split(name, WDICT, neg_per_writer=10, pos_per_writer=10, seed=42):
    """간단한 베이스라인 split 생성 (50/50 균형)"""
    rng = np.random.default_rng(seed)
    writer_ids = list_writer_ids(WDICT)
    bags, labels, metadata = [], [], []
    
    print(f"  {name} split 생성 중... (Writers: {len(writer_ids)})")
    
    # Negative bags (단일 작성자)
    for wid in writer_ids:
        for _ in range(neg_per_writer):
            bag, metas, authors = make_negative_bag(wid, WDICT[wid], rng)
            bags.append(bag)
            labels.append(0)
            metadata.append({
                'authors': authors, 
                'bag_type': 'negative',
                'instances': metas
            })
    
    # Positive bags (복수 작성자, 각 A당 pos_per_writer개, 파트너는 랜덤)
    for widA in writer_ids:
        for _ in range(pos_per_writer):
            widB = rng.choice([w for w in writer_ids if w != widA])
            bag, metas, authors = make_positive_bag(widA, widB, WDICT[widA], WDICT[widB], rng)
            bags.append(bag)
            labels.append(1)
            metadata.append({
                'authors': authors, 
                'bag_type': 'positive',
                'instances': metas
            })
    
    # 전체 셔플
    idx = rng.permutation(len(labels))
    bags = [bags[i] for i in idx]
    labels = [int(labels[i]) for i in idx]
    metadata = [metadata[i] for i in idx]
    
    # 요약 출력
    n_pos = sum(labels)
    n_tot = len(labels)
    print(f"    → Total: {n_tot}, Positive: {n_pos} ({n_pos/n_tot*100:.1f}%), Negative: {n_tot-n_pos}")
    
    # 간단 검증
    assert len(bags) == len(labels) == len(metadata), "Length mismatch"
    assert bags[0].shape == (INSTANCES_PER_BAG, WIN, embed_dim), f"Shape mismatch: {bags[0].shape}"
    
    return bags, labels, metadata

print("✓ Split 생성 함수 정의 완료")

✓ Split 생성 함수 정의 완료


In [6]:
# 베이스라인 Bag 생성 실행
NEG_PW = 10  # 작성자별 negative bags
POS_PW = 10  # 작성자별 positive bags → 전체 약 50/50

print("🔄 베이스라인 Bags 생성 시작...")
print(f"설정: Negative={NEG_PW}/writer, Positive={POS_PW}/writer")

# 모든 split 생성
train_bags, train_labels, train_meta = generate_split('Train', train_writers, NEG_PW, POS_PW, seed=SEED_BASE+0)
val_bags,   val_labels,   val_meta   = generate_split('Val',   val_writers,   NEG_PW, POS_PW, seed=SEED_BASE+10)
test_bags,  test_labels,  test_meta  = generate_split('Test',  test_writers,  NEG_PW, POS_PW, seed=SEED_BASE+20)

# 저장 함수
def save_split(bags, labels, meta, tag, compat_copy=True):
    base = os.path.join(bags_dir, f"bags_arcface_margin_{margin_value}_50p_baseline_{tag}.pkl")
    with open(base, 'wb') as f:
        pickle.dump({'bags': bags, 'labels': labels, 'metadata': meta}, f)
    print(f"💾 저장: {os.path.basename(base)}")
    
    if compat_copy:
        # Stage3 호환을 위한 복사본 (기존 파일명)
        alias = os.path.join(bags_dir, f"bags_arcface_margin_{margin_value}_50p_random_{tag}.pkl")
        with open(alias, 'wb') as f:
            pickle.dump({'bags': bags, 'labels': labels, 'metadata': meta}, f)
        print(f"↪️  호환 복사: {os.path.basename(alias)}")

# 저장 실행
print("\\n💾 파일 저장 중...")
save_split(train_bags, train_labels, train_meta, 'train', compat_copy=True)
save_split(val_bags,   val_labels,   val_meta,   'val',   compat_copy=True)
save_split(test_bags,  test_labels,  test_meta,  'test',  compat_copy=True)

# 최종 요약
def summarize(name, labels):
    n = len(labels)
    p = sum(labels)
    print(f"{name}: N={n}, Pos={p} ({p/n*100:.1f}%), Neg={n-p}")

print("\\n📊 최종 요약:")
summarize('Train', train_labels)
summarize('Val',   val_labels)
summarize('Test',  test_labels)

print("\\n✅ Stage 2 베이스라인 생성 완료!")
print("📋 레이블 정보:")
print("  - Label 0 (Negative): 단일 작성자 (진짜)")
print("  - Label 1 (Positive): 복수 작성자 (위조)")
print("🔗 Stage 3에서 기존 파일명으로 로드 가능")

🔄 베이스라인 Bags 생성 시작...
설정: Negative=10/writer, Positive=10/writer
  Train split 생성 중... (Writers: 180)
    → Total: 3600, Positive: 1800 (50.0%), Negative: 1800
  Val split 생성 중... (Writers: 60)
    → Total: 1200, Positive: 600 (50.0%), Negative: 600
  Test split 생성 중... (Writers: 60)
    → Total: 1200, Positive: 600 (50.0%), Negative: 600
\n💾 파일 저장 중...
💾 저장: bags_arcface_margin_0.4_50p_baseline_train.pkl
↪️  호환 복사: bags_arcface_margin_0.4_50p_random_train.pkl
💾 저장: bags_arcface_margin_0.4_50p_baseline_val.pkl
↪️  호환 복사: bags_arcface_margin_0.4_50p_random_val.pkl
💾 저장: bags_arcface_margin_0.4_50p_baseline_test.pkl
↪️  호환 복사: bags_arcface_margin_0.4_50p_random_test.pkl
\n📊 최종 요약:
Train: N=3600, Pos=1800 (50.0%), Neg=1800
Val: N=1200, Pos=600 (50.0%), Neg=600
Test: N=1200, Pos=600 (50.0%), Neg=600
\n✅ Stage 2 베이스라인 생성 완료!
📋 레이블 정보:
  - Label 0 (Negative): 단일 작성자 (진짜)
  - Label 1 (Positive): 복수 작성자 (위조)
🔗 Stage 3에서 기존 파일명으로 로드 가능


In [7]:
# 간단한 검증 및 샘플 확인

# 데이터 타입과 형태 검증
print("🔍 생성된 데이터 검증:")
print(f"  - Train bags shape: {np.array(train_bags).shape}")
print(f"  - Train labels 분포: {np.bincount(train_labels)}")
print(f"  - 첫 번째 bag shape: {train_bags[0].shape}")
print(f"  - Embedding 차원: {train_bags[0].shape[2]}")

# 샘플 메타데이터 확인
print(f"\\n📋 샘플 메타데이터:")
neg_sample = next(meta for meta, label in zip(train_meta, train_labels) if label == 0)
pos_sample = next(meta for meta, label in zip(train_meta, train_labels) if label == 1)

print(f"  Negative bag:")
print(f"    - Authors: {neg_sample['authors']} (개수: {len(neg_sample['authors'])})")
print(f"    - Type: {neg_sample['bag_type']}")
print(f"    - Instances: {len(neg_sample['instances'])}")

print(f"  Positive bag:")
print(f"    - Authors: {pos_sample['authors']} (개수: {len(pos_sample['authors'])})")
print(f"    - Type: {pos_sample['bag_type']}")
print(f"    - Instances: {len(pos_sample['instances'])}")
print(f"    - 첫 인스턴스 작성자들: {pos_sample['instances'][0]['writer_ids']}")

print(f"\\n✅ 모든 검증 통과 - 베이스라인 데이터 준비 완료!")
print(f"🚀 Stage 3에서 AB-MIL 학습을 진행할 수 있습니다.")

🔍 생성된 데이터 검증:
  - Train bags shape: (3600, 10, 5, 256)
  - Train labels 분포: [1800 1800]
  - 첫 번째 bag shape: (10, 5, 256)
  - Embedding 차원: 256
\n📋 샘플 메타데이터:
  Negative bag:
    - Authors: [64] (개수: 1)
    - Type: negative
    - Instances: 10
  Positive bag:
    - Authors: [95, 13] (개수: 2)
    - Type: positive
    - Instances: 10
    - 첫 인스턴스 작성자들: [13, 13, 95, 95, 95]
\n✅ 모든 검증 통과 - 베이스라인 데이터 준비 완료!
🚀 Stage 3에서 AB-MIL 학습을 진행할 수 있습니다.


In [8]:
# 베이스라인 완료 - 복잡한 설정들은 제거됨

print("=" * 60)
print("🎯 Stage 2 베이스라인 완료!")
print("=" * 60)
print("✅ 간소화된 특징:")
print("  • 단일 규칙: Negative=1작성자, Positive=2작성자 7+7 셔플")
print("  • 고정 윈도우: (5, 1) → 10 인스턴스")
print("  • 50/50 균형 보장")
print()
print("📁 생성된 파일:")
print(f"  • bags_arcface_margin_{margin_value}_50p_baseline_*.pkl (새 이름)")
print(f"  • bags_arcface_margin_{margin_value}_50p_random_*.pkl (Stage3 호환)")
print()
print("🚀 다음 단계: Stage 3에서 AB-MIL 학습")

🎯 Stage 2 베이스라인 완료!
✅ 간소화된 특징:
  • 단일 규칙: Negative=1작성자, Positive=2작성자 7+7 셔플
  • 고정 윈도우: (5, 1) → 10 인스턴스
  • 50/50 균형 보장

📁 생성된 파일:
  • bags_arcface_margin_0.4_50p_baseline_*.pkl (새 이름)
  • bags_arcface_margin_0.4_50p_random_*.pkl (Stage3 호환)

🚀 다음 단계: Stage 3에서 AB-MIL 학습
